In [ ]:
import boto3
import pandas as pd
from sklearn.model_selection import train_test_split

In [10]:
bucket_name = 'demo-bucket-905418222357'
data_file = 'twitter_data.csv'

In [11]:
# Tải dữ liệu từ S3
s3_client = boto3.client('s3')
s3_client.download_file(bucket_name, data_file, data_file)

data = pd.read_csv('twitter_data.csv')

# Chọn các tính năng và nhãn
features = data[['followers_count', 'following_count', 'text_length', 'likes', 'comments']]
labels = data['retweets']

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)


In [5]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Khởi tạo và huấn luyện mô hình
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Dự đoán và đánh giá mô hình
predictions = model.predict(X_test)
mse = mean_squared_error(y_test, predictions)
print(f'Mean Squared Error: {mse}')


Mean Squared Error: 68137654.36210266


In [6]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import r2_score, mean_absolute_error

# Cross-Validation
cv_scores = cross_val_score(model, X_train, y_train, cv=5)
print(f'Cross-Validation Scores: {cv_scores}')
print(f'Mean Cross-Validation Score: {cv_scores.mean()}')

# Đánh giá trên tập kiểm tra
r2 = r2_score(y_test, predictions)
mae = mean_absolute_error(y_test, predictions)
print(f'R^2 Score: {r2}')
print(f'Mean Absolute Error: {mae}')


Cross-Validation Scores: [0.44231974 0.93112842 0.89790761 0.42743765 0.90713999]
Mean Cross-Validation Score: 0.721186681908998
R^2 Score: 0.5478185958451838
Mean Absolute Error: 3436.0944736842107


In [7]:
from sklearn.model_selection import GridSearchCV

# Định nghĩa các hyperparameters cần tối ưu hóa
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10]
}

# Grid Search
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

# Lấy hyperparameters tốt nhất và đánh giá mô hình
best_model = grid_search.best_estimator_
best_predictions = best_model.predict(X_test)
best_mse = mean_squared_error(y_test, best_predictions)
print(f'Best Mean Squared Error: {best_mse}')


Fitting 3 folds for each of 36 candidates, totalling 108 fits
Best Mean Squared Error: 72533271.54160629


## Lưu và triển khai mô hình

In [13]:
import joblib

# Lưu mô hình
joblib.dump(best_model, 'best_model.joblib')

# Upload mô hình lên S3
s3_client.upload_file('best_model.joblib', bucket_name, 'best_model.joblib')

In [ ]:
# Dự đoán số lượt chia sẻ cho bài đăng mới
new_post = {
    'followers_count': [1500],
    'following_count': [500],
    'text_length': [200],
    'likes': [400],
    'comments': [350],
}
new_post_df = pd.DataFrame(new_post)

# Dự đoán
predicted_retweets = model.predict(new_post_df)
print(f'Dự đoán số lượt chia sẻ: {predicted_retweets[0]}')